In [1]:
import category_encoders as ce
import numpy as np
from IPython.display import display
from read_train import read_train
from schema import PREDICTOR_COLS
from schema import PROTECTED_GROUP_COL
from schema import TARGET_COLS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier


 # Random seed

In [2]:
np.random.seed(1337)


 # Extracting targets and predictors from train.csv

In [3]:
predictor_cols = list(PREDICTOR_COLS)
target_cols = list(TARGET_COLS)
train = read_train(target_cols)
train.dropna(subset=target_cols, inplace=True)
train.fillna(-1, inplace=True)

display(train)

X = train[predictor_cols]
y = train[PROTECTED_GROUP_COL]


,High_Performer,Protected_Group,Retained,SJ_Most_1,SJ_Least_1,SJ_Time_1,SJ_Most_2,SJ_Least_2,SJ_Time_2,SJ_Most_3,...,PScale11_Q4,PScale12_Q1,PScale12_Q2,PScale12_Q3,PScale12_Q4,PScale13_Q1,PScale13_Q2,PScale13_Q3,PScale13_Q4,PScale13_Q5
UNIQUE_ID,,,,,,,,,,,,,,,,,,,,,
245021089,0.0,0.0,1,3.0,4.0,95.0,3.0,2.0,101.0,1.0,...,1.0,1.0,1.0,3.0,4.0,1.0,2.0,3.0,2.0,1.0
245181465,1.0,1.0,0,3.0,4.0,53.0,3.0,2.0,57.0,1.0,...,1.0,1.0,1.0,4.0,4.0,2.0,1.0,4.0,4.0,4.0
229682665,0.0,1.0,0,2.0,4.0,89.0,1.0,4.0,73.0,1.0,...,4.0,1.0,1.0,4.0,4.0,1.0,1.0,4.0,4.0,4.0
245174982,1.0,0.0,1,2.0,1.0,110.0,1.0,2.0,86.0,1.0,...,2.0,1.0,1.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0
244979030,0.0,0.0,1,3.0,2.0,35.0,3.0,2.0,21.0,1.0,...,4.0,2.0,1.0,4.0,4.0,2.0,1.0,4.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245107296,0.0,0.0,1,3.0,4.0,84.0,1.0,2.0,52.0,4.0,...,1.0,3.0,2.0,1.0,3.0,2.0,1.0,3.0,3.0,3.0
245038202,0.0,0.0,0,3.0,1.0,36.0,3.0,2.0,20.0,1.0,...,4.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0
242495754,1.0,1.0,1,2.0,4.0,47.0,4.0,2.0,96.0,2.0,...,3.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,3.0


 # Train test split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    stratify=y,
)


 # Pipeline and training

In [5]:
estimator = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    n_estimators=1000,
    learning_rate=0.05,
    n_jobs=4,
)

pipeline = Pipeline(
    steps=[
        ("counter", ce.CountEncoder(cols=predictor_cols)),
        ("estimator", estimator),
    ],
)
pipeline.fit(X_train, y_train)
y_proba = pipeline.predict_proba(X_test)


 # Evaluation

In [6]:
y_pred = np.argmax(y_proba, axis=1)
display(accuracy_score(y_test, y_pred))
display(confusion_matrix(y_test, y_pred))


0.714828897338403

array([[425,  79],
       [146, 139]])